Ejercicio de twitter. Encontrar los 5 estados más felices de USA de acuerdo al análisis de sentimiento con AFFINN.

Contenido. un fichero .ipynb con enunciado. Si no se hace completo indicad en un comentario lo que se ha conseguido Mostrar mapa USA con los estados coloreados por felicidad

Realizar un mapa de España que permita ver dónde se tuitea y en qué idioma se tuitea


In [104]:
import json
import csv
states = ["ak","al","ar","az","ca","co","ct","de","fl","ga","hi","ia","id","il",
          "in","ks","ky","la","ma","md","me","mi","mn","mo","ms","mt","nc","nd","ne","nh",
          "nj","nm","nv","ny","oh","ok","or","pa","ri","sc","sd","tn","tx","ut","va","vt",
          "wa","wi","wv","wy"]

states_names = {'AL': 'Alabama', 'AK': 'Alaska', 'AZ': 'Arizona', 'AR': 'Arkansas', 'CA': 'California', 'CO': 'Colorado',
'CT': 'Connecticut', 'DE': 'Delaware', 'FL': 'Florida', 'GA': 'Georgia', 'HI': 'Hawaii', 'ID': 'Idaho', 'IL': 'Illinois',
'IN': 'Indiana', 'IA': 'Iowa', 'KS': 'Kansas', 'KY': 'Kentucky', 'LA': 'Louisiana', 'ME': 'Maine', 'MD': 'Maryland',
'MA': 'Massachusetts', 'MI': 'Michigan', 'MN': 'Minnesota', 'MS': 'Mississippi', 'MO': 'Missouri', 'MT': 'Montana',
'NE': 'Nebraska', 'NV': 'Nevada', 'NH': 'New Hampshire', 'NJ': 'New Jersey', 'NM': 'New Mexico', 'NY': 'New York', 'NC':'North Carolina', 'ND': 'North Dakota', 'OH': 'Ohio', 'OK': 'Oklahoma', 'OR': 'Oregon', 'PA': 'Pennsylvania', 'RI': 'Rhode Island',
'SC': 'South Carolina', 'SD': 'South Dakota', 'TN':'Tennessee', 'TX': 'Texas', 'UT': 'Utah', 'VT': 'Vermont', 'VA': 'Virginia',
'WA': 'Washington','WV': 'West Virginia', 'WI': 'Wisconsin', 'WY': 'Wyoming'}


In [196]:
#Saving the tweets

from tweepy.streaming import StreamListener
from tweepy import OAuthHandler
from tweepy import Stream
import sys
from jsonmerge import merge

consumer_key = "qDqiLQoTgQusOGa9x0YJJtXr8"
consumer_secret = "HBb2eOxW1XbO1ERIul0UR0zN3MDNz4P1IF6UKqA1qjdXcSBVL4"
 
access_token  = "2792071239-0ewaqKRyFglUAPDjDEjEEikn292OE1rjjFVR9cN"
access_token_secret  = 'xnQimJigJSReMjoCF0WmMpO2nz0dSmjKbTxyUpkg23LJL'

counter = 0
data_dump = {}


#This is a basic listener that just prints received tweets to stdout.
class StdOutListener(StreamListener):
    def on_data(self, data):
        #reading 500 tweets and saving them as valid json to output.txt
        #this took 3 hours. Thanks, JSON
        
        global counter
        counter = counter+1
        
        if counter == 1:
            with open("output.txt", "a+", encoding="utf-8") as outfile:
                outfile.write("["+data+",")
        elif counter == 10000:
            with open("output.txt", "a+", encoding="utf-8") as outfile:
                outfile.write(data+"]")
                sys.exit()
        else:
            with open("output.txt", "a+", encoding="utf-8") as outfile:
                   outfile.write(data+",")

    
    def on_error(self, status):
        print(status)

if __name__ == '__main__':
    counter = 0
    #This handles Twitter authentification and the connection to Twitter Streaming API
    l = StdOutListener()
    auth = OAuthHandler(consumer_key, consumer_secret)
    auth.set_access_token(access_token, access_token_secret)
    stream = Stream(auth, l)

    #This line filter tweets with geoposition .
    stream.filter(locations=[-180,-90,180,90])

SystemExit: 

In [203]:
#reading the tweet data

def getState(data):
    if data["place"] != None and data["place"]["country_code"] == "US":
        state = str(data["place"]["full_name"]).lower().split(", ")
        if len(state) > 1:
            return state[1]

def isState(state):
    if state in states:
        return True
    return False 


with open('output.txt') as json_file:
    tweets = json.load(json_file)
    
sent_dict = {}

#reading the tweet data, check for states and non-empty tweets, 
#add those to sent_dict, where a key is a state and the value
#an array of the tweets
def readJsons():
    for entry in tweets:
        if "created_at" in entry:
            state = getState(entry)
            if isState(state):
                if "text" in entry:
                    if state not in sent_dict.keys():
                        sent_dict[state] = [entry["text"]]
                    else:
                        sent_dict[state].append(entry["text"])

                        
#now we analyze those tweets
from afinn import Afinn
afinn = Afinn()
def evaluateDict():
    score_dict = dict.fromkeys(sent_dict.keys())
    for entry in sent_dict:
        #scoring each set of tweets from each state
        total = 0
        for tweet in sent_dict[entry]:
            #calculating the normalized happiness for the tweets
            score = afinn.score(tweet)/len(sent_dict[entry])
            total = total + score
        score_dict[entry] = total
    #add the states without tweets with a happiness value of 0
    for state in states:
        if state not in score_dict.keys():
            score_dict[state] = 0
    #and returning the 5 happiest states as well as the score_dict
    return sorted(score_dict, key=score_dict.get, reverse=True)[:5],score_dict

In [204]:
#saving the results and sorting them

readJsons()
result = evaluateDict()
print("happiest five states are "+str(result[0]))

#sorting the keys descending and readding the values so that
#the map can be populated properly
score_dict = result[1]
buffer = {}
for key in sorted(score_dict):
    buffer[key]=score_dict[key]
score_dict = buffer   

happiest five states are ['ky', 'sc', 'vt', 'mo', 'wa']


In [206]:
#painting the USA graph
import plotly.graph_objects as go
import plotly
global score_dict

# Load data frame and tidy it.
import pandas as pd
df = pd.read_csv('https://raw.githubusercontent.com/plotly/datasets/master/2011_us_ag_exports.csv')

#generating the plit
fig = go.Figure(data=go.Choropleth(
    locations=df['code'], # Spatial coordinates
    z = list(score_dict.values()),# our sentiment data
    locationmode = 'USA-states', # set of locations match entries in `locations`
    colorscale = 'YlOrRd',
    colorbar_title = "Happiness according to Afinn",
))

#adding layout
fig.update_layout(
    title_text = 'Happiness of US States by Sentiment Analysis of Tweets',
    geo_scope='usa', # limite map scope to USA
)

fig.show()